In [1]:
import pandas as pd
import numpy as np

In [2]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [3]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [4]:
target = []
for i in range(700):
    if data_preprocessed['Absenteeism Time in Hours'][i] >3.0 :
        target.append(1)
    else:
        target.append(0)


In [5]:
data_preprocessed['Exessive Absenteeism'] = target

In [6]:
target.count(1)/(target.count(1)+target.count(0))

0.45571428571428574

In [8]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'],axis=1)
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Exessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,0


In [9]:
unscaled_inputs = data_with_targets.iloc[:,:-1]


# Standardize Inputs

In [10]:
from sklearn.preprocessing import StandardScaler
absenteeism_scaler = StandardScaler()


In [11]:
absenteeism_scaler.fit(unscaled_inputs)
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

# Split Data in train and test

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs,target,test_size=0.15)

# Regression

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [15]:
reg = LogisticRegression()
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
reg.score(x_train,y_train)

0.7647058823529411

In [17]:
reg.intercept_


array([-0.228341])

In [18]:
reg.coef_

array([[ 2.02224061,  0.28017925,  1.5138498 ,  1.35653662,  0.0948959 ,
        -0.19132554,  0.7267374 , -0.05877783, -0.20290736, -0.02767891,
         0.22582934, -0.0529093 ,  0.44206848, -0.3790025 ]])

In [19]:
summary_table = pd.DataFrame(columns=['Feature Name'], data = unscaled_inputs.columns.values)

In [20]:
summary_table['Coefficients'] = np.transpose(reg.coef_)

In [21]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['intercept',reg.intercept_[0]]

In [22]:
summary_table = summary_table.sort_index()
summary_table['Odd_ratio'] = np.exp(summary_table['Coefficients'])

In [23]:
summary_table

,Feature Name,Coefficients,Odd_ratio
0,intercept,-0.228341,0.795853
1,Reason_1,2.022241,7.555234
2,Reason_2,0.280179,1.323367
3,Reason_3,1.513850,4.544191
4,Reason_4,1.356537,3.882723
5,Month,0.094896,1.099544
6,Day of week,-0.191326,0.825864
7,Transportation Expense,0.726737,2.068321
8,Distance to Work,-0.058778,0.942916
9,Age,-0.202907,0.816354


In [24]:
import pickle

In [25]:
with open('model','wb') as file:
    pickle.dump(reg,file)


In [26]:
with open('Preprocess_abstenteeism','wb') as file:
    pickle.dump(scaled_inputs,file)